In [1]:
import pathlib
import pandas as pd
from scipy.stats import ttest_ind, ttest_rel
import seaborn as sns

In [2]:
clinical = pd.read_csv(pathlib.Path('discovery_EV_microarray/discovery_full_design.csv'))
clinical

,Unnamed: 0,Batch,PatientID_cleaned,SampleName,PatientID.TubeNumber,Gender,Sample.Type,TimePoint,real_time,Days_relative_aPD1,...,Progression_aPD1,PD1_PFS,Vital_status..1.dead..0.alive.,Adverse_event,Has_metadata,Steroids,Days_aPD1_last_contact,XRT_or_Resection_during_aPD1,aPD1_tx_length.1,batch
0,0077_SEP_A1_r_0077_SEP_A2_r_P4_0018210A_A01,100077,208,PretxRNA208-1,208,Female,Tissue,3/11/15,pre,0,...,1,79,1,NoAdverseEvent,1,NoSteroids,521,0.0,127.0,100077
1,0077_SEP_A10_r_0077_SEP_A9_r_P4_0018210A_A09,100077,42,PretxRNA42-2,42,Male,Tissue,10/17/14,pre,0,...,1,47,1,NoAdverseEvent,1,NoSteroids,163,0.0,61.0,100077
2,0077_SEP_A5_r_0077_SEP_A6_r_P4_0018210A_A05,100077,236,PretxRNA236-1,236,Male,Tissue,11/2/15,pre,-10,...,1,335,0,AdverseEvent,1,Steroids,670,1.0,670.0,100077
3,0077_SEP_B10_r_0077_SEP_B11_r_P4_0018210A_B11,100077,156,PretxRNA156-1,156,Male,Tissue,5/13/15,pre,0,...,0,607,1,AdverseEvent,1,Steroids,607,0.0,52.0,100077
4,0077_SEP_B2_r_0077_SEP_B3_r,100077,379,PretxRNA379-1,379,Male,Tissue,7/9/15,pre,-1,...,1,84,1,NoAdverseEvent,1,NoSteroids,144,0.0,84.0,100077
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
103,P4_0029405A_C03,100453,416,NaN,416,Male,PBMC,9/15/15,pre,0,...,1,85,1,NoAdverseEvent,1,NoSteroids,283,0.0,66.0,100453
104,P4_0029405A_C04,100453,BI1,NaN,BI1,Male,PBMC,9/11/15,pre,-38,...,1,60,1,NoAdverseEvent,1,NaN,159,NaN,NaN,100453
105,P4_0029405A_C09,100453,178,NaN,178,Male,Exosomes,7/14/14,pre,0,...,0,1165,0,NoAdverseEvent,1,Steroids,1165,0.0,497.0,100453
106,P4_0029405A_C10,100453,324,NaN,324,Male,Exosomes,4/29/15,pre,0,...,0,599,1,NoAdverseEvent,1,Steroids,599,0.0,252.0,100453


In [3]:
clinical.columns

Index(['Unnamed: 0', 'Batch', 'PatientID_cleaned', 'SampleName',
       'PatientID.TubeNumber', 'Gender', 'Sample.Type', 'TimePoint',
       'real_time', 'Days_relative_aPD1', 'aPD1_tx_length',
       'Patient.Condition',
       'Response..0.noresponse..1.response..2.mixed.response..4..unknown.',
       'binary_response', 'PatientID_cleaned.1', 'Cell_line', 'Is_Tumor',
       'Cell.line', 'Control', 'Exosomes', 'Tissue', 'Time_series', 'pre',
       'Time', 'Tx_Start', 'Time_since_start', 'Unnamed..23', 'has_prior_tx',
       'Tx0', 'Tx0_date_start', 'Tx0_resp', 'Tx1', 'Tx1_date_start',
       'Tx1_resp', 'Tx2', 'Tx2_date_start', 'Tx2_resp',
       'has_prior_immunotherapy', 'has_prior_other_therapy', 'Tx3',
       'Tx3_date_start', 'Tx3_resp', 'Clinical_Benefit', 'Prior_immunotherapy',
       'Progression_aPD1', 'PD1_PFS', 'Vital_status..1.dead..0.alive.',
       'Adverse_event', 'Has_metadata', 'Steroids', 'Days_aPD1_last_contact',
       'XRT_or_Resection_during_aPD1', 'aPD1_tx_leng

In [4]:
clinical = clinical[[
    'Unnamed: 0', 'PatientID_cleaned', 'Clinical_Benefit', 'Exosomes', 
    'Sample.Type', 'pre', 'real_time'
]]
clinical

,Unnamed: 0,PatientID_cleaned,Clinical_Benefit,Exosomes,Sample.Type,pre,real_time
0,0077_SEP_A1_r_0077_SEP_A2_r_P4_0018210A_A01,208,NoClinicalBenefit,Tis,Tissue,pre,pre
1,0077_SEP_A10_r_0077_SEP_A9_r_P4_0018210A_A09,42,NoClinicalBenefit,Tis,Tissue,pre,pre
2,0077_SEP_A5_r_0077_SEP_A6_r_P4_0018210A_A05,236,ClinicalBenefit,Tis,Tissue,pre,pre
3,0077_SEP_B10_r_0077_SEP_B11_r_P4_0018210A_B11,156,ClinicalBenefit,Tis,Tissue,pre,pre
4,0077_SEP_B2_r_0077_SEP_B3_r,379,NoClinicalBenefit,Tis,Tissue,pre,pre
...,...,...,...,...,...,...,...
103,P4_0029405A_C03,416,NoClinicalBenefit,Tis,PBMC,pre,pre
104,P4_0029405A_C04,BI1,NoClinicalBenefit,Tis,PBMC,pre,pre
105,P4_0029405A_C09,178,ClinicalBenefit,Exo,Exosomes,pre,pre
106,P4_0029405A_C10,324,ClinicalBenefit,Exo,Exosomes,pre,pre


In [6]:
log = pd.read_csv(pathlib.Path('discovery_EV_microarray/discovery_log2_eset.csv'))
log

,Unnamed: 0,0077_SEP_A1_r_0077_SEP_A2_r_P4_0018210A_A01,0077_SEP_A10_r_0077_SEP_A9_r_P4_0018210A_A09,0077_SEP_A5_r_0077_SEP_A6_r_P4_0018210A_A05,0077_SEP_B10_r_0077_SEP_B11_r_P4_0018210A_B11,0077_SEP_B2_r_0077_SEP_B3_r,0077_SEP_B6_r_0077_SEP_B7_r_P4_0018210A_B07,0077_SEP_C2_r_0077_SEP_C3_r_P4_0018210A_C02,0077_SEP_C6_r_0077_SEP_C7_r,P4_0018210A_A01,...,P4_0029405A_B10,P4_0029405A_B11,P4_0029405A_B12,P4_0029405A_C01,P4_0029405A_C02,P4_0029405A_C03,P4_0029405A_C04,P4_0029405A_C09,P4_0029405A_C10,P4_0029405A_C11
0,DDX11L1,5.533981,5.954646,5.612661,6.088217,6.690878,5.457135,7.196882,5.308425,4.727423,...,6.361140,6.211320,5.731874,6.742823,4.706090,7.288164,5.381614,6.950132,7.737552,4.343521
1,WASH7P,5.042716,6.844405,7.441109,5.815662,4.842598,6.188048,6.822621,4.593950,5.344774,...,5.860542,7.005711,5.513957,6.996831,7.471100,7.228444,7.228444,4.145790,4.449621,4.425250
2,OR4F5,1.508885,1.967562,2.089677,1.517301,2.146420,1.222544,1.879208,0.330193,1.125716,...,1.438267,1.661697,1.143540,1.706171,2.254137,1.296270,2.029032,1.295338,0.965942,2.119076
3,FAM41C,3.835130,3.511898,4.710639,3.727020,3.628411,3.552289,4.854446,2.325988,4.000201,...,4.623152,3.944908,3.324469,2.976824,4.494470,4.541520,4.884296,4.261451,2.299992,1.579987
4,SAMD11,4.412451,4.372109,3.770265,4.358718,4.270727,4.520660,5.632778,3.934607,4.468575,...,4.115289,4.231225,4.773176,4.719023,4.466712,4.591435,3.836840,5.150260,5.654435,6.216162
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21224,RBMY2FP,2.414530,1.391941,1.758824,0.894464,1.743423,1.856667,1.860384,1.288704,2.104129,...,2.210422,2.108918,1.854845,1.005200,2.196039,1.605271,0.781307,0.870402,3.501566,3.660642
21225,PRY,2.515487,2.280625,2.456231,2.441390,2.789060,2.673675,4.498935,2.476737,2.508234,...,2.106101,3.193130,3.158935,2.970041,2.115470,2.768617,1.494131,4.350151,4.556334,4.404640
21226,DAZ4,2.164390,1.514162,2.465532,1.892171,2.882211,2.236754,2.015977,1.964917,1.478310,...,2.615481,2.296399,1.614695,1.938852,1.997299,2.723150,1.200235,3.723365,2.178748,3.088962
21227,CSPG4P1Y,3.257206,2.612168,2.663164,2.858013,3.443211,3.829588,3.696074,2.280625,3.598631,...,3.357848,3.122601,3.575698,2.710482,1.945125,4.932671,3.191290,3.918179,5.381982,4.267577


In [7]:
log.index = log['Unnamed: 0']
log = log.drop('Unnamed: 0', 1)
log

,0077_SEP_A1_r_0077_SEP_A2_r_P4_0018210A_A01,0077_SEP_A10_r_0077_SEP_A9_r_P4_0018210A_A09,0077_SEP_A5_r_0077_SEP_A6_r_P4_0018210A_A05,0077_SEP_B10_r_0077_SEP_B11_r_P4_0018210A_B11,0077_SEP_B2_r_0077_SEP_B3_r,0077_SEP_B6_r_0077_SEP_B7_r_P4_0018210A_B07,0077_SEP_C2_r_0077_SEP_C3_r_P4_0018210A_C02,0077_SEP_C6_r_0077_SEP_C7_r,P4_0018210A_A01,P4_0018210A_A05,...,P4_0029405A_B10,P4_0029405A_B11,P4_0029405A_B12,P4_0029405A_C01,P4_0029405A_C02,P4_0029405A_C03,P4_0029405A_C04,P4_0029405A_C09,P4_0029405A_C10,P4_0029405A_C11
Unnamed: 0,,,,,,,,,,,,,,,,,,,,,
DDX11L1,5.533981,5.954646,5.612661,6.088217,6.690878,5.457135,7.196882,5.308425,4.727423,4.976730,...,6.361140,6.211320,5.731874,6.742823,4.706090,7.288164,5.381614,6.950132,7.737552,4.343521
WASH7P,5.042716,6.844405,7.441109,5.815662,4.842598,6.188048,6.822621,4.593950,5.344774,7.111312,...,5.860542,7.005711,5.513957,6.996831,7.471100,7.228444,7.228444,4.145790,4.449621,4.425250
OR4F5,1.508885,1.967562,2.089677,1.517301,2.146420,1.222544,1.879208,0.330193,1.125716,1.922601,...,1.438267,1.661697,1.143540,1.706171,2.254137,1.296270,2.029032,1.295338,0.965942,2.119076
FAM41C,3.835130,3.511898,4.710639,3.727020,3.628411,3.552289,4.854446,2.325988,4.000201,4.617020,...,4.623152,3.944908,3.324469,2.976824,4.494470,4.541520,4.884296,4.261451,2.299992,1.579987
SAMD11,4.412451,4.372109,3.770265,4.358718,4.270727,4.520660,5.632778,3.934607,4.468575,3.732926,...,4.115289,4.231225,4.773176,4.719023,4.466712,4.591435,3.836840,5.150260,5.654435,6.216162
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
RBMY2FP,2.414530,1.391941,1.758824,0.894464,1.743423,1.856667,1.860384,1.288704,2.104129,1.934781,...,2.210422,2.108918,1.854845,1.005200,2.196039,1.605271,0.781307,0.870402,3.501566,3.660642
PRY,2.515487,2.280625,2.456231,2.441390,2.789060,2.673675,4.498935,2.476737,2.508234,2.501378,...,2.106101,3.193130,3.158935,2.970041,2.115470,2.768617,1.494131,4.350151,4.556334,4.404640
DAZ4,2.164390,1.514162,2.465532,1.892171,2.882211,2.236754,2.015977,1.964917,1.478310,1.999647,...,2.615481,2.296399,1.614695,1.938852,1.997299,2.723150,1.200235,3.723365,2.178748,3.088962


In [8]:
pbmc_benefit = clinical[
    (clinical['Sample.Type'] == 'PBMC ') & (clinical['Clinical_Benefit'] == 'ClinicalBenefit')]
pbmc_benefit

,Unnamed: 0,PatientID_cleaned,Clinical_Benefit,Exosomes,Sample.Type,pre,real_time
96,P4_0029405A_B08,178,ClinicalBenefit,Tis,PBMC,pre,pre
97,P4_0029405A_B09,236,ClinicalBenefit,Tis,PBMC,pre,pre
98,P4_0029405A_B10,307,ClinicalBenefit,Tis,PBMC,pre,pre
99,P4_0029405A_B11,324,ClinicalBenefit,Tis,PBMC,pre,pre
102,P4_0029405A_C02,409,ClinicalBenefit,Tis,PBMC,pre,pre


In [9]:
pbmc_nobenefit = clinical[
    (clinical['Sample.Type'] == 'PBMC ') & (clinical['Clinical_Benefit'] == 'NoClinicalBenefit')]
pbmc_nobenefit

,Unnamed: 0,PatientID_cleaned,Clinical_Benefit,Exosomes,Sample.Type,pre,real_time
100,P4_0029405A_B12,331,NoClinicalBenefit,Tis,PBMC,pre,pre
101,P4_0029405A_C01,379,NoClinicalBenefit,Tis,PBMC,pre,pre
103,P4_0029405A_C03,416,NoClinicalBenefit,Tis,PBMC,pre,pre
104,P4_0029405A_C04,BI1,NoClinicalBenefit,Tis,PBMC,pre,pre


In [10]:
log[pbmc_benefit['Unnamed: 0']]

,P4_0029405A_B08,P4_0029405A_B09,P4_0029405A_B10,P4_0029405A_B11,P4_0029405A_C02
Unnamed: 0,,,,,
DDX11L1,4.968957,6.555369,6.361140,6.211320,4.706090
WASH7P,5.294476,6.723236,5.860542,7.005711,7.471100
OR4F5,1.992482,2.805610,1.438267,1.661697,2.254137
FAM41C,4.787975,4.670508,4.623152,3.944908,4.494470
SAMD11,3.807120,3.850118,4.115289,4.231225,4.466712
...,...,...,...,...,...
RBMY2FP,1.068078,1.764912,2.210422,2.108918,2.196039
PRY,2.642803,2.711966,2.106101,3.193130,2.115470
DAZ4,2.688212,1.310898,2.615481,2.296399,1.997299


In [12]:
data = {'gene': [], 'statistic': [], 'pvalue': []}
for i in log[pbmc_benefit['Unnamed: 0']].index:
    t = ttest_ind(log[pbmc_benefit['Unnamed: 0']].loc[i], log[pbmc_nobenefit['Unnamed: 0']].loc[i])
    if t[1] < 0.05:
        data['gene'].append(i)
        data['statistic'].append(t[0])
        data['pvalue'].append(t[1])

df = pd.DataFrame(data=data)

In [13]:
df

,gene,statistic,pvalue
0,KLHL17,-3.012544,0.019592
1,PERM1,-3.056100,0.018424
2,C1orf159,-2.682196,0.031439
3,ACAP3,-2.681544,0.031469
4,MXRA8,-2.499978,0.040994
...,...,...,...
1695,RENBP,-3.174640,0.015607
1696,MIR3202-1,2.501890,0.040879
1697,MIR3202-2,4.567398,0.002582
1698,F8,2.476115,0.042451
